# Questo script ha lo scopo di addestrare sullo stesso dataset alcuni classificatori classici e stamparne le stesse metriche di valutazione usate per il caso quantistico.

In [ ]:
# Installazione delle librerie necessarie
!pip install sklearn
!pip install pandas
!pip install seaborn


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
import time
from sklearn.datasets import load_wine
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import (accuracy_score, precision_score, recall_score,
                           f1_score, confusion_matrix, classification_report)
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression

# _Stessa logica di gestione del caso precedente, main centralizzato che gestisce il tutto_

In [ ]:
# Impostazioni per i plot
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

def main():
    """
    Funzione main che esegue tutto il pipeline di classificazione classica
    """

    # =============================================================================
    # 1. CARICAMENTO E PREPROCESSING DEL DATASET WINE CON FEATURE SELECTION
    # =============================================================================


    print("=" * 80)
    print("CARICAMENTO DATASET WINE E PREPROCESSING CON PCA")
    print("=" * 80)

    # Caricamento dataset Wine
    wine_data = load_wine()

    features = wine_data.data
    labels = wine_data.target
    feature_names = wine_data.feature_names
    target_names = wine_data.target_names

    # Standardizzazione per PCA 
    print("\n" + "-" * 50)
    print("STANDARDIZZAZIONE E PCA")
    print("-" * 50)

    scaler = StandardScaler()
    features_scaled = scaler.fit_transform(features)


    pca = PCA(n_components=5)
    features_pca = pca.fit_transform(features_scaled)


    # Normalizzazione delle componenti PCA per i circuiti quantistici
    normalizer = MinMaxScaler()
    features_pca5_normalized = normalizer.fit_transform(features_pca)


    # Split train/test
    train_features, test_features, train_labels, test_labels = train_test_split(
        features_pca5_normalized, labels, train_size=0.8, random_state=123, stratify=labels
    )

    # =============================================================================
    # 2. ESECUZIONE ESPERIMENTI CLASSICI
    # =============================================================================

    print(f"\n" + "=" * 80)
    print("AVVIO ESPERIMENTI CLASSIFICATORI CLASSICI")
    print("=" * 80)
    print(f"Iniziando esperimenti con {features_pca5_normalized.shape[1]} features")
    print(f"Dataset: {len(train_features)} train samples, {len(test_features)} test samples")
    print(f"Classi da predire: {len(target_names)} ({', '.join(target_names)})")

    classical_results = run_classical_experiments(
        train_features, train_labels, test_features, test_labels, target_names
    )

    # =============================================================================
    # 3. ANALISI COMPLETA DEI RISULTATI
    # =============================================================================

    print("\n" + "=" * 80)
    print("ANALISI FINALE CLASSIFICATORI CLASSICI")
    print("=" * 80)

    # Analisi dei risultati classici
    analyze_classical_results(classical_results)

    print("\n" + "=" * 80)
    print("ESPERIMENTI CLASSICI COMPLETATI CON SUCCESSO!")
    print("=" * 80)

    return classical_results


## _Funzione che avvia gli esperimenti classici sfruttando 6 calssificatori_

In [ ]:

def run_classical_experiments(train_features, train_labels, test_features, test_labels, target_names):
    """Esegue tutti gli esperimenti comparativi con classificatori classici"""

    print("\n" + "=" * 80)
    print("ESPERIMENTI COMPARATIVI CLASSIFICATORI CLASSICI CON METRICHE DETTAGLIATE")
    print("=" * 80)

    # Definiamo i classificatori classici
    classifiers = {
        'Random Forest': RandomForestClassifier(n_estimators=100, random_state=123),
        'Support Vector Machine': SVC(kernel='rbf', random_state=123, probability=True),
        'K-Nearest Neighbors': KNeighborsClassifier(n_neighbors=5),
        'Gradient Boosting': GradientBoostingClassifier(random_state=123),
        'Logistic Regression': LogisticRegression(random_state=123, max_iter=1000),
        'Naive Bayes': GaussianNB()
    }

    results = []
    experiment_count = 0
    total_experiments = len(classifiers)

    print(f"Totale esperimenti da eseguire: {total_experiments}")
    print(f"Numero di features per esperimento: {train_features.shape[1]}")
    print("-" * 80)

    for clf_name, classifier in classifiers.items():
        experiment_count += 1
        print(f"\n{'='*20} ESPERIMENTO {experiment_count}/{total_experiments} {'='*20}")
        print(f"Classificatore: {clf_name}")

        # Stampa dei parametri del modello se disponibili
        if hasattr(classifier, 'get_params'):
            params = classifier.get_params()
            key_params = []
            if 'n_estimators' in params:
                key_params.append(f"n_estimators={params['n_estimators']}")
            if 'kernel' in params:
                key_params.append(f"kernel={params['kernel']}")
            if 'n_neighbors' in params:
                key_params.append(f"n_neighbors={params['n_neighbors']}")
            if key_params:
                print(f"Parametri principali: {', '.join(key_params)}")

        # Addestramento
        print("Addestramento in corso...")
        start_time = time.time()

        classifier.fit(train_features, train_labels)

        training_time = time.time() - start_time

        # Valutazione
        train_pred = classifier.predict(train_features)
        test_pred = classifier.predict(test_features)

        train_acc = accuracy_score(train_labels, train_pred)
        test_acc = accuracy_score(test_labels, test_pred)

        # Calcolo metriche dettagliate
        test_precision = precision_score(test_labels, test_pred, average='weighted')
        test_recall = recall_score(test_labels, test_pred, average='weighted')
        test_f1 = f1_score(test_labels, test_pred, average='weighted')

        print(f"RISULTATI:")
        print(f"Train Accuracy: {train_acc:.4f}")
        print(f"Test Accuracy:  {test_acc:.4f}")
        print(f"Test Precision: {test_precision:.4f}")
        print(f"Test Recall:    {test_recall:.4f}")
        print(f"Test F1-Score:  {test_f1:.4f}")
        print(f"Training Time:  {training_time:.4f}s")

        # Confusion Matrix per questo esperimento
        experiment_title = f"{clf_name}"
        print(f"\n CONFUSION MATRIX - {experiment_title}")

        cm = confusion_matrix(test_labels, test_pred)
        plt.figure(figsize=(8, 6))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                    xticklabels=target_names, yticklabels=target_names)
        plt.title(f'Confusion Matrix\n{experiment_title}')
        plt.xlabel('Predicted')
        plt.ylabel('True')
        plt.show()

        # Classification report dettagliato
        print(f"\n CLASSIFICATION REPORT - {experiment_title}")
        print(classification_report(test_labels, test_pred, target_names=target_names))

        # Salvataggio risultati
        result = {
            'classifier': clf_name,
            'train_accuracy': train_acc,
            'test_accuracy': test_acc,
            'test_precision': test_precision,
            'test_recall': test_recall,
            'test_f1': test_f1,
            'training_time': training_time,
            'success': True
        }

        results.append(result)

    print("-" * 80)

    return results


# _Funzione che plotta e analizza i risultati ottenuti e ricevuti dalla funzione precedente_

In [ ]:

def analyze_classical_results(classical_results):
    """Analizza e visualizza i risultati degli esperimenti classici"""

    print("\n" + "=" * 80)
    print("ANALISI AVANZATA DEI RISULTATI CLASSICI")
    print("=" * 80)

    # Conversione in DataFrame
    df_results = pd.DataFrame(classical_results)

    # Statistiche generali
    print("\n1. STATISTICHE GENERALI")
    print("-" * 40)
    successful_experiments = df_results[df_results['success'] == True]
    print(f"Esperimenti riusciti: {len(successful_experiments)}/{len(df_results)}")
    print(f"Accuracy media (train): {successful_experiments['train_accuracy'].mean():.4f} ± {successful_experiments['train_accuracy'].std():.4f}")
    print(f"Accuracy media (test): {successful_experiments['test_accuracy'].mean():.4f} ± {successful_experiments['test_accuracy'].std():.4f}")
    print(f"Precision media (test): {successful_experiments['test_precision'].mean():.4f} ± {successful_experiments['test_precision'].std():.4f}")
    print(f"Recall media (test): {successful_experiments['test_recall'].mean():.4f} ± {successful_experiments['test_recall'].std():.4f}")
    print(f"F1-Score media (test): {successful_experiments['test_f1'].mean():.4f} ± {successful_experiments['test_f1'].std():.4f}")
    print(f"Tempo medio di training: {successful_experiments['training_time'].mean():.4f}s ± {successful_experiments['training_time'].std():.4f}s")

    # Top configurazioni per test accuracy
    print("\n2. RANKING CLASSIFICATORI (Test Accuracy)")
    print("-" * 80)
    top_configs = successful_experiments.sort_values('test_accuracy', ascending=False)
    print(f"{'Rank':<4} {'Classifier':<25} {'Test Acc':<8} {'Precision':<9} {'Recall':<7} {'F1':<7} {'Time(s)':<8}")
    print("-" * 80)
    for rank, (idx, row) in enumerate(top_configs.iterrows(), 1):
        print(f"{rank:<4} {row['classifier']:<25} {row['test_accuracy']:<8.4f} {row['test_precision']:<9.4f} "
              f"{row['test_recall']:<7.4f} {row['test_f1']:<7.4f} {row['training_time']:<8.4f}")

    # Grafico comparativo delle metriche
    if len(successful_experiments) > 0:
        plt.figure(figsize=(15, 10))

        # Subplot per accuracy
        plt.subplot(2, 3, 1)
        plt.bar(successful_experiments['classifier'], successful_experiments['test_accuracy'], color='skyblue')
        plt.title('Test Accuracy per Classificatore')
        plt.ylabel('Accuracy')
        plt.xticks(rotation=45, ha='right')
        plt.grid(True, alpha=0.3)

        # Subplot per precision
        plt.subplot(2, 3, 2)
        plt.bar(successful_experiments['classifier'], successful_experiments['test_precision'], color='lightgreen')
        plt.title('Test Precision per Classificatore')
        plt.ylabel('Precision')
        plt.xticks(rotation=45, ha='right')
        plt.grid(True, alpha=0.3)

        # Subplot per recall
        plt.subplot(2, 3, 3)
        plt.bar(successful_experiments['classifier'], successful_experiments['test_recall'], color='salmon')
        plt.title('Test Recall per Classificatore')
        plt.ylabel('Recall')
        plt.xticks(rotation=45, ha='right')
        plt.grid(True, alpha=0.3)

        # Subplot per F1-score
        plt.subplot(2, 3, 4)
        plt.bar(successful_experiments['classifier'], successful_experiments['test_f1'], color='gold')
        plt.title('Test F1-Score per Classificatore')
        plt.ylabel('F1-Score')
        plt.xticks(rotation=45, ha='right')
        plt.grid(True, alpha=0.3)

        # Subplot per training time
        plt.subplot(2, 3, 5)
        plt.bar(successful_experiments['classifier'], successful_experiments['training_time'], color='plum')
        plt.title('Training Time per Classificatore')
        plt.ylabel('Tempo (secondi)')
        plt.xticks(rotation=45, ha='right')
        plt.grid(True, alpha=0.3)

        # Subplot combinato per tutte le metriche
        plt.subplot(2, 3, 6)
        x = np.arange(len(successful_experiments))
        width = 0.15

        plt.bar(x - 2*width, successful_experiments['test_accuracy'], width, label='Accuracy', alpha=0.8)
        plt.bar(x - width, successful_experiments['test_precision'], width, label='Precision', alpha=0.8)
        plt.bar(x, successful_experiments['test_recall'], width, label='Recall', alpha=0.8)
        plt.bar(x + width, successful_experiments['test_f1'], width, label='F1-Score', alpha=0.8)

        plt.title('Confronto Tutte le Metriche')
        plt.ylabel('Score')
        plt.xticks(x, successful_experiments['classifier'], rotation=45, ha='right')
        plt.legend()
        plt.grid(True, alpha=0.3)

        plt.tight_layout()
        plt.show()

    # Miglior classificatore
    if len(successful_experiments) > 0:
        best_classifier = successful_experiments.loc[successful_experiments['test_accuracy'].idxmax()]

        print(f"\n MIGLIOR CLASSIFICATORE:")
        print(f"   Nome: {best_classifier['classifier']}")
        print(f"   Test Accuracy: {best_classifier['test_accuracy']:.4f}")
        print(f"   Precision: {best_classifier['test_precision']:.4f}")
        print(f"   Recall: {best_classifier['test_recall']:.4f}")
        print(f"   F1-Score: {best_classifier['test_f1']:.4f}")
        print(f"   Training Time: {best_classifier['training_time']:.4f}s")

    return successful_experiments

In [ ]:
# Esecuzione del main
if __name__ == "__main__":
    classical_results = main()